In [1]:
embedding_size = 256
hidden_size = 128
batch_size = 32
output_size = 10 # It is known that Willow Cove features 10 "ports"
num_heads = 6

learning_rate = 0.001

In [2]:
import model.utils
import torch
from torch_geometric.loader import DataLoader
import torch.utils.data

dataset = model.utils.BasicBlockDataset("data/i5_1135g7.pb", dtype=torch.float32)
train_size = int(0.3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [3]:
from model.GraphEncoder import GATEncoder
from model.Predictor import Predictor
import torch_geometric

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

encoder = GATEncoder(dataset.num_opcodes, embedding_size, hidden_size, num_heads).to(device)
model = Predictor(encoder, hidden_size, output_size, batch_size).to(device)

In [4]:
from model.model import train

checkpoint_dir = "checkpoints/tgl"
checkpoint_freq = 10
num_epochs = 200

if torch.cuda.is_available():
    torch.cuda.empty_cache()

train(model, device, loader, num_epochs, batch_size, learning_rate, checkpoint_dir, checkpoint_freq)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | GATEncoder | 4.8 M 
1 | fc      | Linear     | 1.3 K 
2 | relu    | ReLU       | 0     
---------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.076    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fe048d53760>
Traceback (most recent call last):
  File "/home/alex/anaconda3/envs/pytorch2/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 161113, 161137, 161161, 161185, 161209) exited unexpectedly

In [9]:
torch.save(model, "trained_models/tgl.pt")

In [11]:
choice = dataset[140]
bb, m, raw = choice

print(bb.x)

input_sequence = bb.x.to(device)
edge_index = bb.edge_index.to(device)

out, _ = model(input_sequence, edge_index)
res = out.to("cpu").detach().numpy()
model.utils.print_port_pressure_table(res[1:], raw["source"])
print(model.utils.estimate_cycles(out))
print(m)